In [1]:
import os
from pathlib import Path
from PIL import Image
from typing import Any, Literal

from prettytable import PrettyTable
import skimage as ski
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchmetrics as tm
from torchvision.models import get_model, get_weight
from torchvision.datasets import DatasetFolder
from torchvision.transforms import v2

# Setup

In [2]:
DATASET_PATHS = {
    'fog-detection': Path('./datasets/fog-detection-dataset-prepared'),
    'fog-or-smog': Path('./datasets/fog-or-smog-detection-dataset-prepared'),
    'foggy-cityscapes': Path('./datasets/foggy-cityscapes-image-dataset-prepared')
}

In [3]:
def load_image(path: str):
    img = ski.io.imread(path)
    if img.ndim == 2:  # Handle grayscale
        img = ski.color.gray2rgb(img)
    if img.shape[-1] == 4:  # Handle RGBA
        img = ski.color.rgba2rgb(img)
    img = ski.util.img_as_ubyte(img)
    img = img.squeeze()
    
    return Image.fromarray(img)

train_transforms = v2.Compose([
    v2.ToImage(),
    # v2.Grayscale(1),
    v2.RandomResizedCrop(size=(224,224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

test_transforms = v2.Compose([
    v2.ToImage(),
    # v2.Grayscale(1),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

def get_dataloader(
    path: Path | str,
    transform: Any | None = None,
    batch_size: int = 32,
    shuffle: bool = False,
    num_workers: int = 0,
    pin_memory: bool = True,
):
    return DataLoader(
        dataset=DatasetFolder(
            root=path,
            loader=load_image,
            extensions=[".jpg", ".png", ".jpeg"],
            transform=transform,
        ),
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=True if num_workers > 0 else False,
    )

In [4]:
dl = get_dataloader(DATASET_PATHS['fog-detection'], train_transforms, batch_size=2)
x, y = next(iter(dl))
x.shape, y.shape

(torch.Size([2, 3, 224, 224]), torch.Size([2]))

In [5]:
class LSTMClassifier(L.LightningModule):
    def __init__(
        self,
        image_width: int,
        image_height: int,
        image_channels: int,
        hidden_size: int,
        num_lstm_layers: int,
        num_classes: int,
        dropout: float = 0.0,
        loss: nn.Module | None = None,
        learning_rate: float= 1e-4,
        optimizer_name: Literal['adam', 'adamw'] = 'adam',
    ):
        super().__init__()
        self.save_hyperparameters()

        self.learning_rate = learning_rate
        self.optimizer_name = optimizer_name

        self.image_height = image_height
        self.image_width = image_width
        self.image_channels = image_channels
        self.hiddes_size = hidden_size
        self.num_lst_layers = num_lstm_layers
        self.lstm_input_size = self.image_channels * self.image_width
        self.num_classes = num_classes
        self.dropout = dropout
        self.model = nn.LSTM(
            input_size=self.lstm_input_size,
            hidden_size=self.hiddes_size,
            num_layers=self.num_lst_layers,
            batch_first=True,
            dropout=self.dropout
        )
        self.fc = nn.Linear(self.hiddes_size, self.num_classes)

        if loss is not None:
            self.criterion= loss
        else:
            self.criterion = nn.CrossEntropyLoss()

        task = "multiclass" if num_classes > 2 else "binary"
        
        self.train_metrics = tm.MetricCollection({
            "accuracy": tm.classification.Accuracy(task=task, num_classes=num_classes),
            "f1": tm.classification.F1Score(task=task, num_classes=num_classes),
            "precision": tm.classification.Precision(task=task, num_classes=num_classes),
            "recall": tm.classification.Recall(task=task, num_classes=num_classes),
        }, prefix="train_")
        self.validation_metrics = self.train_metrics.clone(prefix="val_")
        self.test_metrics = self.train_metrics.clone(prefix="test_")

    def forward(self, x):
        batch_size = x.size(0)

        x = x.permute(0, 2, 1, 3)
        x = x.reshape(batch_size, self.image_height, self.lstm_input_size)

        lstm_out, (hn, cn) = self.model(x)
        last_time_step_out= lstm_out[:, -1, :]
        
        return self.fc(last_time_step_out)
    
    def _common_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, labels
    
    def training_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)

        self.log("train_loss", loss)
        self.log_dict(self.train_metrics(preds, labels))

        return loss
    
    def on_train_epoch_end(self):
        self.train_metrics.reset()

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.validation_metrics.update(preds, labels)
        self.log("val_loss", loss)
        return loss
    
    def on_validation_epoch_end(self):
        self.log_dict(self.validation_metrics.compute())
        self.validation_metrics.reset()
    

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.test_metrics.update(preds, labels)
        return loss
    
    def on_test_epoch_end(self):
        self.log_dict(self.test_metrics.compute())
        self.test_metrics.reset()

    def configure_optimizers(self):
        if self.optimizer_name == 'adam':
            optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_name == 'adamw':
            optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}.")
        return optimizer

# Training

## Normalized

In [6]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR')
MODEL_NAME = "LSTM"
DATASET = 'fog-detection'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR/LSTM-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         Non-trainable params

Epoch 33: 100%|██████████| 12/12 [00:05<00:00,  2.32it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8301886916160583     │
│          test_f1          │    0.8615384697914124     │
│      test_precision       │    0.8615384697914124     │
│        test_recall        │    0.8615384697914124     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7974137663841248     │
│          test_f1          │    0.8252788186073303     │
│      test_precision       │     0.744966447353363     │
│        test_recall        │     0.925000011920929     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.56it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5454545617103577     │
│          test_f1          │    0.4642857015132904     │
│      test_precision       │    0.5652173757553101     │
│        test_recall        │    0.39393940567970276    │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.8302,0.8615,0.8615,0.8615
fog-or-smog,0.7974,0.8253,0.745,0.925
foggy-cityscapes,0.5455,0.4643,0.5652,0.3939


In [7]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR')
MODEL_NAME = "LSTM"
DATASET = 'fog-or-smog'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR/LSTM-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         Non-trainable params
5

Epoch 33: 100%|██████████| 52/52 [00:05<00:00,  8.83it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:02<00:00,  1.46it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6320754885673523     │
│          test_f1          │    0.6138613820075989     │
│      test_precision       │    0.8611111044883728     │
│        test_recall        │    0.4769230782985687     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.931034505367279     │
│          test_f1          │     0.931034505367279     │
│      test_precision       │    0.9642857313156128     │
│        test_recall        │    0.8999999761581421     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:13<00:00,  0.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5959596037864685     │
│          test_f1          │    0.48051947355270386    │
│      test_precision       │    0.6727272868156433     │
│        test_recall        │    0.3737373650074005     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6321,0.6139,0.8611,0.4769
fog-or-smog,0.931,0.931,0.9643,0.9
foggy-cityscapes,0.596,0.4805,0.6727,0.3737


In [8]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR')
MODEL_NAME = "LSTM"
DATASET = 'foggy-cityscapes'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR/LSTM-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         Non-trainable par

Epoch 6: 100%|██████████| 22/22 [00:13<00:00,  1.59it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6415094137191772     │
│          test_f1          │    0.6984127163887024     │
│      test_precision       │    0.7213114500045776     │
│        test_recall        │    0.6769230961799622     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7780172228813171     │
│          test_f1          │    0.8110091686248779     │
│      test_precision       │    0.7245901823043823     │
│        test_recall        │    0.9208333492279053     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5909090638160706     │
│          test_f1          │    0.5803108811378479     │
│      test_precision       │    0.5957446694374084     │
│        test_recall        │    0.5656565427780151     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6415,0.6984,0.7213,0.6769
fog-or-smog,0.778,0.811,0.7246,0.9208
foggy-cityscapes,0.5909,0.5803,0.5957,0.5657


## No normalization

In [9]:
train_transforms = v2.Compose([
    v2.ToImage(),
    # v2.Grayscale(1),
    v2.RandomResizedCrop(size=(224,224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=[0.5], std=[0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

test_transforms = v2.Compose([
    v2.ToImage(),
    # v2.Grayscale(1),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=[0.5], std=[0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

In [10]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'fog-detection'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION/LSTM-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         Non

Epoch 25: 100%|██████████| 12/12 [00:05<00:00,  2.29it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7358490824699402     │
│          test_f1          │    0.8082191944122314     │
│      test_precision       │    0.7283950448036194     │
│        test_recall        │    0.9076923131942749     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.756465494632721     │
│          test_f1          │    0.7971274852752686     │
│      test_precision       │    0.7003154754638672     │
│        test_recall        │     0.925000011920929     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5101010203361511     │
│          test_f1          │    0.6711864471435547     │
│      test_precision       │    0.5051020383834839     │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.7358,0.8082,0.7284,0.9077
fog-or-smog,0.7565,0.7971,0.7003,0.925
foggy-cityscapes,0.5101,0.6712,0.5051,1.0


In [11]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'fog-or-smog'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION/LSTM-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         Non-t

Epoch 14: 100%|██████████| 52/52 [00:06<00:00,  8.33it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.18it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6698113083839417     │
│          test_f1          │    0.6666666865348816     │
│      test_precision       │           0.875           │
│        test_recall        │    0.5384615659713745     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9224137663841248     │
│          test_f1          │    0.9220778942108154     │
│      test_precision       │    0.9594594836235046     │
│        test_recall        │     0.887499988079071     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.56it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.631313145160675     │
│          test_f1          │    0.5034013390541077     │
│      test_precision       │    0.7708333134651184     │
│        test_recall        │    0.3737373650074005     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6698,0.6667,0.875,0.5385
fog-or-smog,0.9224,0.9221,0.9595,0.8875
foggy-cityscapes,0.6313,0.5034,0.7708,0.3737


In [12]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'foggy-cityscapes'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-COLOR-NO-NORMALIZATION/LSTM-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 542 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
542 K     Trainable params
0         

Epoch 8: 100%|██████████| 22/22 [00:13<00:00,  1.69it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:02<00:00,  1.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.650943398475647     │
│          test_f1          │    0.7448275685310364     │
│      test_precision       │     0.675000011920929     │
│        test_recall        │    0.8307692408561707     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7176724076271057     │
│          test_f1          │    0.7798319458961487     │
│      test_precision       │    0.6535211205482483     │
│        test_recall        │    0.9666666388511658     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:11<00:00,  0.61it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5707070827484131     │
│          test_f1          │    0.6931408047676086     │
│      test_precision       │    0.5393258333206177     │
│        test_recall        │    0.9696969985961914     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6509,0.7448,0.675,0.8308
fog-or-smog,0.7177,0.7798,0.6535,0.9667
foggy-cityscapes,0.5707,0.6931,0.5393,0.9697
